## Define Variables / Import MetaData

In [ ]:
import os
import sys
from pathlib import Path

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.dates as mdates
from cartopy import crs as ccrs 
import cartopy.feature as cfeature
import pandas as pd
import hvplot.pandas
import xarray as xr
import hvplot.xarray
import geoviews.feature as gf
from sklearn.metrics import root_mean_squared_error

## Functions

In [ ]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
from utils.functions import import_flux_metadata, import_flux_site_data, convert_flux_to_micasa_units, replace_outliers_with_nan, clean_flux_datasets 

## Define variables 

In [ ]:
# I can't get the config.py to work in jupyternotebook because it does not know where $NOBACKUP is
amer_filepath = '../../ameriflux-data/'
mic_filepath = '../preprocessing/intermediates/'

In [ ]:
FLUX_DATA_PATH = Path(amer_filepath)
FLUX_METADATA = Path(FLUX_DATA_PATH, "AmeriFlux-site-search-results-202410071335.tsv")
MICASA_PREPROCESSED_DATA = Path(mic_filepath)

In [ ]:
timedelta = "DD"

In [ ]:
# set map proj
proj=ccrs.PlateCarree()

## Plotting

In [ ]:
# Define site ID
# site_ID = 'CA-MA1'
# site_ID = 'CA-DBB'
# site_ID = 'CA-LP1'
# site_ID = 'AR-TF1'
# site_ID = 'BR-CST'
site_ID = 'US-Wi3'

In [ ]:
fluxnet_sel = import_flux_site_data(FLUX_DATA_PATH, site_ID, timedelta)
fluxnet_sel

In [ ]:
cols = fluxnet_sel.columns.tolist()
list = [cols[0], cols[-1]]
list

In [ ]:
new_list = ["NEE (kgC m-2 s-1)", "GPP_DT (kgC m-2 s-1)"]

In [ ]:
for old, new in zip(list, new_list):
   fluxnet_sel = convert_flux_to_micasa_units(fluxnet_sel, old, new)
   fluxnet_sel = clean_flux_datasets(fluxnet_sel, new, "NEE_VUT_REF_QC") 

In [ ]:
fluxnet_sel

In [ ]:
# Mask GPP outliers
fluxnet_sel = replace_outliers_with_nan(fluxnet_sel, "GPP_DT (kgC m-2 s-1)")

In [ ]:
############ Import Preprocessed Micasa Data ################
filename = f"{site_ID}_micasa_{timedelta}.csv"
path = os.path.join(MICASA_PREPROCESSED_DATA, filename)
micasa_ds = pd.read_csv(path, index_col=0, parse_dates=True)

############## Append datasets #########################
# Make clean dataframe and append together
## NEE
NEE_ds = pd.DataFrame()
NEE_ds["MiCASA"] = micasa_ds["MiCASA NEE (kg m-2 s-1)"]
NEE_ds["FluxNet"] = fluxnet_sel["NEE (kgC m-2 s-1)"]

In [ ]:
NEE_ds.plot()

In [ ]:
# NPP
NPP_ds = pd.DataFrame()
NPP_ds["MiCASA"] = micasa_ds["MiCASA NPP (kg m-2 s-1)"]
NPP_ds["FluxNet DT GPP/2"] = fluxnet_sel["GPP_DT (kgC m-2 s-1)"] / 2

In [ ]:
NPP_ds.plot()

## Site Info

In [ ]:
fluxnet_meta = import_flux_metadata(FLUX_METADATA)
site_lat = fluxnet_meta.loc[
    fluxnet_meta["Site ID"] == site_ID, "Latitude (degrees)"
].values
site_lon = fluxnet_meta.loc[
    fluxnet_meta["Site ID"] == site_ID, "Longitude (degrees)"
].values

In [ ]:
# Define subset site info to display
site_subset = ['Site ID', 
                'Name', 
                'Vegetation Description (IGBP)', 
                'Climate Class Description (Koeppen)', 
                'Elevation (m)',
                'Years of AmeriFlux FLUXNET Data']

In [ ]:
site_sel = fluxnet_meta.loc[fluxnet_meta['Site ID'] == site_ID][site_subset]
with pd.option_context('display.max_colwidth', None):
  display(site_sel)

## Create final static plots

In [ ]:
# Create a subplot grid with specific width ratios
fig, axs = plt.subplots(4, 1, 
                         gridspec_kw={'height_ratios': [1, 2,0.25,2],
                                      'hspace': 0.01},
                         figsize=(10, 12)) 

# Define the map projection
proj = ccrs.PlateCarree()


if site_lat >= 20:
    # North America extents
    min_lon, max_lon = -170, -57
    min_lat, max_lat = 25, 74

else:
    # South America extents
    min_lon, max_lon = -90, -30
    min_lat, max_lat = -60, 12
axs[0].axis('off')
axs[0] = plt.subplot(4, 1, 1, projection=proj,frameon=False)
axs[0].set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())
# axs[0].add_feature(cfeature.STATES)
# axs[0].add_feature(cfeature.BORDERS)
axs[0].coastlines()

axs[0].scatter(site_lon,site_lat,
       marker='*', 
       s=500,
       color='yellow',
       edgecolor='black',
               zorder=3)

NEE_ds.plot(ax=axs[1],ylabel = 'NEE\n(kgC m$^{-2}$ s$^{-1}$)')
# Set pretty date labels
axs[1].xaxis.set_major_locator(mdates.AutoDateLocator())
# Disable minor ticks completely
axs[1].tick_params(axis='x', which='minor', labelsize=0, labelcolor='none')
# axs[1].xaxis.set_major_formatter(mdates.ConciseDateFormatter(axs[3].xaxis.get_major_locator()))

axs[2].set_visible(False)

NPP_ds.plot(ax=axs[3],ylabel = 'NPP\n(kgC m$^{-2}$ s$^{-1}$)')
# Set pretty date labels
axs[3].xaxis.set_major_locator(mdates.AutoDateLocator())
# Disable minor ticks completely
axs[3].tick_params(axis='x', which='minor', labelsize=0, labelcolor='none')



date_format = mdates.DateFormatter('%b %Y')
for i in range(1,4,2):
    axs[i].xaxis.set_major_formatter(date_format)
    axs[i].set_xlabel('') 
fig.suptitle(f'{site_ID}',y=0.9,fontsize=14)

# RSME

## Calc RSME

In [ ]:
# Drop NA values!
NEE_ds_clean = NEE_ds.dropna(subset=['FluxNet'])
NPP_ds_clean = NPP_ds.dropna(subset=['FluxNet DT GPP/2'])

In [ ]:
NEE_RSME = root_mean_squared_error(NEE_ds_clean.MiCASA, NEE_ds_clean.FluxNet)
NEE_RSME

In [ ]:
NPP_RSME = root_mean_squared_error(NPP_ds_clean.MiCASA, NPP_ds_clean["FluxNet DT GPP/2"])

## RSME plotting

In [ ]:
# Create df with lat/lons
site_subset = ['Site ID', 
               'Longitude (degrees)',
                'Latitude (degrees)',
               ]
df_meta = fluxnet_meta[site_subset]
df_meta.set_index('Site ID')

In [ ]:
# Import and merge results
results = pd.read_csv('../analysis/results_no_mask.csv',index_col='SiteID')
results

In [ ]:
df = df_meta.join(results, on='Site ID')
df

In [ ]:
ds = xr.Dataset(
    coords={
        'site_id': df['Site ID'].values,
        'lat': ('site_id', df['Latitude (degrees)'].values),
        'lon': ('site_id', df['Longitude (degrees)'].values),
    }, 
    data_vars={
        'NEE_RSME': ("site_id", df['NEE_RSME'].values),
        'NPP_RSME': ("site_id", df['NPP_RSME'].values),
    }
)

In [ ]:
ds

### Xarray matplotlib

In [ ]:
cmap = plt.cm.autumn_r
cmap

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(12, 10), subplot_kw={'projection': proj}, constrained_layout=True);
fig.suptitle('MiCASA, FluxNet Sites Root Mean Squared Error (RSME)', y=0.76)
values =['NEE_RSME', 'NPP_RSME']
for ax,val in zip(axs, values):
    ax.add_feature(cfeature.COASTLINE,zorder=0)
    plot = ds.plot.scatter(x="lon", y="lat",ax=ax,
                           
                           markersize=val, edgecolor='none',add_legend=False,
                           
                            norm=colors.LogNorm(), 
                            # norm=colors.LogNorm(vmin=ds[val].min(), vmax=ds[val].max()),
                           hue=val,
                           cmap='autumn_r',
                           # cmap='cool',
                           add_colorbar=False
                          )
    
    cbar = fig.colorbar(plot, ax=ax, shrink=0.9, label=val[4:], orientation='horizontal')
    ax.set_title(val[:3])
plt.show()

### Make custom colormap with transparency

In [ ]:
# from matplotlib.colors import ListedColormap

In [ ]:
# Make transparency colormap:
cmap = plt.cm.autumn_r
cmap

In [ ]:
# cmap(np.arange(cmap.N)).shape

In [ ]:
cmap(1)

In [ ]:
# my_cmap = cmap(np.arange(cmap.N))
# my_cmap[:, -1] = np.linspace(0, 1, cmap.N)
# my_cmap = ListedColormap(my_cmap)
# my_cmap

In [ ]:
my_cmap = ListedColormap(my_cmap)
my_cmap

In [ ]:
# fig, axs = plt.subplots(1,2,figsize=(12, 10), subplot_kw={'projection': proj}, constrained_layout=True);
# fig.suptitle('MiCASA, FluxNet Sites Root Mean Squared Error (RSME)', y=0.76)
# values =['NEE_RSME', 'NPP_RSME']
# for ax,val in zip(axs, values):
#     ax.add_feature(cfeature.COASTLINE,zorder=0)
#     plot = ds.plot.scatter(x="lon", y="lat",ax=ax,
                           
#                            markersize=val, edgecolor='none',add_legend=False,
                           
#                             norm=colors.LogNorm(), 
#                             # norm=colors.LogNorm(vmin=ds[val].min(), vmax=ds[val].max()),
#                            hue=val,
#                            cmap=my_cmap,
#                            add_colorbar=False
#                           )
    
#     cbar = fig.colorbar(plot, ax=ax, shrink=0.9, label=val[4:], orientation='horizontal')
#     ax.set_title(val[:3])
# plt.show()

In [ ]:
%matplotlib ipympl
ds.plot.scatter(x="lon", y="lat")

#### Xarray bokeh plot? This doesn't work so I have to plot the dataframe

In [ ]:
# ds_NEE = ds[values[0]]
# ds_NEE

In [ ]:
# hv.extension('bokeh', inline=True)
# ds_NEE.hvplot.points(x='lon', y='lat',
#                       geo=True,
#                      # crs=proj, 
#                     # project=True
#                      )

# ds_dropped = ds_NEE.drop_indexes("site_id")
# ds_dropped = ds_dropped.drop_vars("site_id")
# ds_dropped

### Matplotlib CONUS only

In [ ]:
lons_conus = [-140, -60]
lats_conus = [20, 60]

In [ ]:
lons_conus +lats_conus

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(16,10), subplot_kw={'projection': proj}, constrained_layout=True);
fig.suptitle('MiCASA, FluxNet Sites Root Mean Squared Error (RSME)', 
             y=0.7,size=16,
            )
values =['NEE_RSME', 'NPP_RSME']
for ax,val in zip(axs, values):
    ax.set_extent(lons_conus +lats_conus)
    ax.add_feature(cfeature.LAND,zorder=0, linewidth=0.3, color='lightgrey')
    ax.add_feature(cfeature.STATES,zorder=0, linewidth=0.3)
    
    plot = ds.plot.scatter(x="lon", y="lat",ax=ax,
                           
                           markersize=val, edgecolor='none',add_legend=False,
                            norm=colors.LogNorm(), 
                           hue=val,
                           cmap='autumn_r',
                           add_colorbar=False
                          )
    
    cbar = fig.colorbar(plot, ax=ax, shrink=0.8, label=val[4:], orientation='horizontal')
    ax.set_title(val[:3])
plt.show()

### Pandas Holoviews

In [ ]:
df.head()

In [ ]:
import xyzservices.providers as xyz
from matplotlib.ticker import LogFormatter

In [ ]:
min_lon, max_lon = df["Longitude (degrees)"].min(), df["Longitude (degrees)"].max()
min_lat, max_lat = df["Latitude (degrees)"].min(), df["Latitude (degrees)"].max()

print(min_lon, max_lon)
print(min_lat, max_lat)

In [ ]:
my_cmap

In [ ]:
plot_list = []
for i, value in enumerate(values): 
    plot = df.hvplot.points(x="Longitude (degrees)", 
                            y="Latitude (degrees)",
                            geo=True, 
                            crs=ccrs.PlateCarree(),
                            # projection=ccrs.PlateCarree(), # Doesn't work with tiles
    
                             #Custom cmap with transparency won't show up in bokeh
                            c=value,
                            logz=True,
                            cmap=my_cmap,
                            clabel=f'{value}',
    
                             size=45,
                             # Size values don't scale logarithmically
                            # s=values[0],
                            # scale=4500,
                             # color='red',
                            
                            tiles=True,
                            tiles_opts={'alpha': 0.4},
                            # tiles=xyz.Esri.WorldGrayCanvas,
    
    
                            hover_cols=['Site ID'],
    
                            # width=700, height=500,
                            xlim=(-170, -20),   # longitude range
                            ylim=(-60, 75),     # latitude range
                            # frame_width=800,
                            frame_height=700
                                               )
    plot_list.append(plot)

In [ ]:
(plot_list[0] * gf.coastline).opts(title="Micasa/Ameriflux Net Ecosystem Exchange (NEE) RSME")

In [ ]:
(plot_list[1] * gf.coastline).opts(title="Micasa/Ameriflux Net Primary Productivity (NPP) RSME")


### Old

#### Try ipympl? It doesn't seem to work

In [ ]:
import ipympl

In [ ]:
%matplotlib ipympl
ds.plot.scatter(x="lon", y="lat")

In [ ]:
# !jupyter labextension list

In [ ]:
%matplotlib inline

In [ ]:
# Try to scale size by log:
# df_scale = df.copy()
# df_scale["log_NEE_RSME"] = np.log(df_scale["NEE_RSME"])
# df_scale.head()

# **** This doesn't work because the logs are negative- would have to create a pseudo log scale but this is complex ******

### Run holoviews with matplotlib backend

In [ ]:
import holoviews as hv
hv.extension('matplotlib')

In [ ]:
df.hvplot.points(x="Longitude (degrees)", 
                        y="Latitude (degrees)",
                        crs=ccrs.PlateCarree(),
                        # frame_width=800,
                        # frame_height=500
                       )

In [ ]:
plot * gf.coastline

### Run a random example

In [ ]:
import numpy as np
import xarray as xr

lat = np.linspace(-90, 90, 5)
lon = np.linspace(-180, 180, 5)
data = np.random.rand(len(lat), len(lon))

ds_ex = xr.DataArray(data, coords=[('lat', lat), ('lon', lon)], name='val')

ds_ex.hvplot.image(x='lon', y='lat')  # no geo=True yet

In [ ]:
ds_ex

In [ ]:
ds_ex.hvplot.points(x="lon", y="lat", 
                 # c=val,
                    geo=True,
                  projection=proj,
                 # coastline=True,
                 )

### Try geopandas?

In [ ]:
# import geopandas as gpd
# from shapely.geometry import Point


In [ ]:
# gdf = gpd.GeoDataFrame(
#     {"site_id": ds["site_id"].values},
#     geometry=[Point(xy) for xy in zip(ds["lon"].values, ds["lat"].values)],
#     crs="EPSG:4326"
# )